In [1]:
import numpy as np
import pandas as pd
import time
from collections import deque

import tensorflow as tf
from six import next
from sklearn import preprocessing
import sys
from scipy.sparse import lil_matrix
from scipy.sparse import coo_matrix

C:\Users\yphy0\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\yphy0\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\yphy0\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\yphy0\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

### movielens 1M

#### User data

전체 6040명 gender, age, occpuation, post 의 4가지 features 존재

In [2]:
col_names = ['user','gender','age','occupation','postcode']
lm_df = pd.read_csv('./data/ml1m/users.dat', sep='::', header=None, names=col_names, engine='python')
del lm_df['postcode'] #대문자 -> 소문자
lm_df["user"]-=1 #1씩 빼서 0부터 시작하게
lm_df=pd.get_dummies(lm_df,columns=[ "age", "gender", "occupation"]) 
del lm_df['user']

In [3]:
lm_df

,age_1,age_18,age_25,age_35,age_45,age_50,age_56,gender_F,gender_M,occupation_0,...,occupation_11,occupation_12,occupation_13,occupation_14,occupation_15,occupation_16,occupation_17,occupation_18,occupation_19,occupation_20
0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
6036,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6037,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6038,0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


#### Item data

user, gender, 

In [112]:
col_names = ["movieid", "movietitle","year", "Genre"]
df = pd.read_csv('./data/ml1m/newmovies.dat', sep='::', header=None, names=col_names, engine='python')
del df['movietitle'] #movieid가 있으니 title 굳이안씀
df["movieid"]-=1
df=pd.concat([df,df['Genre'].str.get_dummies(sep='|').add_prefix('Genre_').astype('int8')],axis=1) #genre개수만큼 columns 추가
del  df["Genre"] #genre 삭제
df['year'] -= df['year'].min() #일종의 정규화
df['year'] /= df['year'].max() #연도도 0~1사이의 값으로 변환. 1에가까울수록 최신
df['year'] = df['year'].fillna(0.0)

#
ITEM_NUM = 3952
ItemGenreMatrix= [[0 for x in range(18)] for y in range(ITEM_NUM)] #장르의 수가 18개, ITEM_NUM X 18 의 values list
ItemYearMatrix= [[0 for x in range(1)] for y in range(ITEM_NUM)] # ITEM_NUM X 1, year value

movieIdList=df["movieid"].values #3883개의 값들
movieGenres=df.drop(['movieid','year'],axis=1 ).values #3883 X 18 -> M X J ?
movieYears = df['year'].values #3883
for indx in range(len(movieIdList)): #IDlist의 index의 값들 불러오기
    mvid = movieIdList[indx] #id
    mvgenre=movieGenres[indx] #genre 1~18의 one-hot vector
    mvyear=movieYears[indx] #years value
    ItemGenreMatrix[mvid]=mvgenre 
    ItemYearMatrix[mvid]=[mvyear] 
    
ItemGenreMatrix = np.asarray(ItemGenreMatrix) #np.asarray는 복사본이 아니라 참조본 생성 , 3952 X 18
ItemYearMatrix = np.asarray(ItemYearMatrix) #3952 X 1
ItemFeatures = np.concatenate((ItemGenreMatrix,ItemYearMatrix),axis =1) #3952 X 19

## like_num, comment_num , post_date(?), style , 

In [116]:
ItemFeatures

array([[0.       , 0.       , 1.       , ..., 0.       , 0.       ,
        0.9382716],
       [0.       , 1.       , 0.       , ..., 0.       , 0.       ,
        0.9382716],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.9382716],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]])

### Movielens 100k

In [98]:
col_names = ["movieid", "movietitle", "releasedate", "videoreleasedate","IMDbURL"
            ,"unknown","Action","Adventure","Animation","Childrens","Comedy","Crime","Documentary"
            ,"Drama","Fantasy","FilmNoir","Horror","Musical","Mystery","Romance","SciFi","Thriller"
            ,"War","Western"]
df = pd.read_csv('./data/ml100k/u.item', sep='|', header=None, names=col_names, engine='python')
df['releasedate'] = pd.to_datetime(df['releasedate'])
df['year'],df['month']=zip(*df['releasedate'].map(lambda x: [x.year,x.month]))
df['year']-=df['year'].min()
df['year']/=df['year'].max()
df['year']=df['year'].fillna(0.0)

del df["month"] #year만빼고버림
del df["movietitle"]
del df["releasedate"]
del df["videoreleasedate"]
del df["IMDbURL"] #url 버림

df["movieid"]-=1
del df["movieid"]
df.values #1682 x 20

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.96052632],
       [0.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.96052632],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.96052632],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.94736842],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.97368421]])

## D

In [33]:
############# ML 100k dataset ###########

BATCH_SIZE = 1000
PERC=0.9
USER_NUM = 943
ITEM_NUM = 1682
df_train, df_test = get_data100k()

# #Without Graph Feature
# MFSIZE=40
# UW=0.08
# IW=0.06
# LR=0.0002
# EPOCH_MAX = 601
# tf.reset_default_graph()
# GraphRec(df_train, df_test,ItemData=False,UserData=False,Graph=False,Dataset='100k')

# #With Graph Features
# MFSIZE=50
# UW=0.05
# IW=0.02
# LR=0.00003
# EPOCH_MAX = 196
# tf.reset_default_graph()
# GraphRec(df_train, df_test,ItemData=False,UserData=False,Graph=True,Dataset='100k')

#############################################


In [15]:
def read_process(filname, sep="\t"):
    col_names = ["user", "item", "rate", "st"]
    df = pd.read_csv(filname, sep=sep, header=None, names=col_names, engine='python')
    df["user"] -= 1
    df["item"] -= 1
    for col in ("user", "item"):
        df[col] = df[col].astype(np.int32)
    df["rate"] = df["rate"].astype(np.float32)
    return df


In [19]:
def get_data100k():
    global PERC
    df = read_process("./data/ml100k/u.data", sep="\t")
    rows = len(df)
    df = df.iloc[np.random.permutation(rows)].reset_index(drop=True)
    split_index = int(rows * PERC)
    df_train = df[0:split_index]
    df_test = df[split_index:].reset_index(drop=True)
    return df_train, df_test

PERC = 0.9

train_df = get_data100k()[0]

In [31]:
train_df

,user,item,rate,st
0,675,750,4.0,892685695
1,757,1089,1.0,882055460
2,536,483,4.0,886031105
3,429,233,4.0,877226323
4,16,8,3.0,885272558
...,...,...,...,...
89995,701,288,2.0,885767604
89996,540,264,5.0,885595654
89997,416,189,5.0,879647065
89998,711,622,4.0,874729778


In [34]:
AdjacencyUsers = np.zeros((USER_NUM,ITEM_NUM), dtype=np.float32) # N x M shape의 zero matrix 생성 (Adjacency)
DegreeUsers = np.zeros((USER_NUM,1), dtype=np.float32) #N x 1  shape의 zero vactor 생성 (Degree)

AdjacencyItems = np.zeros((ITEM_NUM,USER_NUM), dtype=np.float32) # M x N shape의 zero matrix 생성
DegreeItems =  np.zeros((ITEM_NUM,1), dtype=np.float32)  # M X 1 shape의 zero vactor 생성

In [35]:
DegreeItems

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [28]:
for index, row in train_df.iterrows():
    userid=int(row['user'])
    itemid=int(row['item'])
    AdjacencyUsers[userid][itemid]=row['rate']/5.0
    AdjacencyItems[itemid][userid]=row['rate']/5.0
    DegreeUsers[userid][0]+=1
    DegreeItems[itemid][0]+=1

In [ ]:
      userid=int(row['user'])
      itemid=int(row['item'])